Thoughts and notes:

The output will be dark matter relic density, which is best understood on a log scale. Hence the loss function should be mean_squared_logarithmic_error rather than regular mean squared error (mse).

Should split data set into about 70% training, 15% validation, and 15% test data.

Weirdness found in EWKino.csv so far:

1) These 9 columns ["mqL1", "muR", "mdR", "meL", "mmuL", "meR", "mmuR", "mtauL", "mtauR"] only contain the value 10,000. This is expected, simply drop them.
2) Both mu.png and M_2.png have identical bands missing from their middle, from about -50 to +50. Why?
3) mA.png has many fewer points close to zero than looks right.
4) At.png looks like it has a lense dense patch in the middle right. Is this deliberate?

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("EWKino.csv")

In [9]:
min_df = df[["M_1", "M_2", "mu", "tan_beta", "M_3", "At", "Ab", "Atau", "mA", "mqL3", "mtR", "mbR"]]
min_df.head()

,M_1,M_2,mu,tan_beta,M_3,At,Ab,Atau,mA,mqL3,mtR,mbR
0,-191.78204,1546.65420,464.17776,21.946095,4467.1616,7964.2593,-996.21590,-1833.5303,4799.62350,3953.6410,4136.0884,4439.0024
1,508.58246,-1485.52060,-845.79706,45.249397,2966.3315,7814.7046,1948.92920,1869.5571,914.76697,2074.7473,3877.8281,3648.5361
2,256.92212,-586.74084,827.61096,44.627483,2525.1646,-7629.4720,-886.47943,1569.5186,3707.04800,4660.5483,3552.7420,2651.8157
3,1414.03080,1839.58840,1488.70280,34.970398,2217.5942,1812.8965,561.58276,-1885.2010,2538.65480,2316.7524,4820.9746,4380.9146
4,907.39870,1121.29700,364.57916,24.084797,4698.1206,-4202.6980,101.66391,-1358.2156,3297.12600,4534.0107,4456.1587,4051.9758


In [ ]:
scaler = StandardScaler()
for column in min_df.columns:
    min_df[column] = scaler.transform(min_df[column])
    # Trying to standardise all features to zero mean unit variance

min_df.head()


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
"""def drop_zero_columns(df):
    # Removes all columns from a DataFrame where every value is zero.
    
    # Get columns where all values are zero
    zero_cols = df.columns[(df == 0).all(axis=0)]
    
    # Drop those columns and return
    return zero_cols, df.drop(columns=zero_cols)"""

In [ ]:
"""def plot_scatter_from_pandas(df, column_name):
    # Extract the values from the specified column
    values = df[column_name]
    
    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(values)), values, s=0.1)
    
    # Add labels and title
    plt.xlabel("Index")
    plt.ylabel(column_name)
    plt.title(f"Scatter Plot of '{column_name}' Column")
    
    # Save the plot
    plt.savefig(f"plots/{column_name}.png")
    
    # Show the plot
    plt.show()"""